In [ ]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-rcubd7dr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-rcubd7dr
  Resolved https://github.com/huggingface/datasets.git to commit 76020180407d7ea9a0b535758d8d1b241fd19d8c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
  Created wheel for datasets: filename=datasets-2.15.1.dev0-py3-none-any.whl size=500112 sha256=3b9856058cddf8d98e62aad3883bec1b2551b86bff946e4fa76eb5f53eda9db9
  Stored in directory: /tmp/pip-ephem-wheel-cache-2rissew3/wheels/57/f4/c4/53c677af89fec0ef3226c1e75a38367b37c2fa626f0544d3e4
Successfully built datasets
  Cloning https://github.com/huggingface/transformers.git to /tmp/p

# **Dataset Preparation**

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")
# df = pd.read_csv('/content/drive/MyDrive/MLProject/lyrics-gpt.csv')
# df = pd.read_csv('/content/drive/MyDrive/MLProject/balanced-lyrics-for-genre.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/MLProject/songs-balanced-final.csv')
import ast
df['merged_labels'] = df['merged_labels'].apply(ast.literal_eval)

In [ ]:
import torchaudio

def load_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    # Additional preprocessing steps if needed
    return waveform, sample_rate


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# all_labels = set()
# for labels in df['merged_labels']:
#     all_labels.update(labels)
all_genres=['electronic', 'rock', 'funk&soul', 'pop', 'folk&country', 'hiphop', 'rhythm&blues', 'jazz']
all_moods = ['happy/celebration/party/dance', 'loss/sad/heartbreak/angst/protest', 'romantic/love/passion/devotion', 'motivating/inspirational/uplifting/confidence/nostalgia']

all_labels_ordered = all_genres + all_moods
mlb = MultiLabelBinarizer(classes=all_labels_ordered)
mlb.fit([all_labels_ordered])

MultiLabelBinarizer(classes=['electronic', 'rock', 'funk&soul', 'pop',
                             'folk&country', 'hiphop', 'rhythm&blues', 'jazz',
                             'happy/celebration/party/dance',
                             'loss/sad/heartbreak/angst/protest',
                             'romantic/love/passion/devotion',
                             'motivating/inspirational/uplifting/confidence/nostalgia'])

In [ ]:
df['encoded_labels'] = df['merged_labels'].apply(lambda x: mlb.transform([x])[0])
df.head()

,merged_labels,base_path,file_path,encoded_labels
0,"[electronic, rock, happy/celebration/party/dance]",rock/-i2Kn-EXh_A#rock#Super Furry Animals - (D...,/content/drive/MyDrive/MLProject/spectrogram-w...,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
1,"[rock, loss/sad/heartbreak/angst/protest]",rock/-JJAXwAaA2w#rock#Placebo - The Bitter End...,/content/drive/MyDrive/MLProject/spectrogram-w...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
2,"[rock, funk&soul, romantic/love/passion/devotion]",rock/-nqRkAsZumc#rock#Incubus - Stellar.webm,/content/drive/MyDrive/MLProject/spectrogram-w...,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
3,"[rock, motivating/inspirational/uplifting/conf...",rock/-Ue193sAcrE#rock#Maximo Park - Apply Some...,/content/drive/MyDrive/MLProject/spectrogram-w...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,"[rock, loss/sad/heartbreak/angst/protest]",rock/-_EprMTJPWA#rock#Idlewild - American Engl...,/content/drive/MyDrive/MLProject/spectrogram-w...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and the rest
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# **Model Preparation - Wav2Vec2 for Multilabel classification**

SRC:- https://www.kaggle.com/code/lujar1762/music-genre-classification-with-wav2vec2/notebook

In [ ]:
model_name_or_path = "m3hrdadfi/wav2vec2-base-100k-gtzan-music-genres"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

import transformers
from transformers import (
    HfArgumentParser,
    TrainingArguments,
    EvalPrediction,
    AutoConfig,
    Wav2Vec2Processor,
    Wav2Vec2FeatureExtractor,
    is_apex_available,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint, is_main_process


import os
import sys
import librosa
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput

In [ ]:
@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [ ]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

In [ ]:
class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [ ]:
model_name_or_path = "facebook/wav2vec2-base-100k-voxpopuli"
pooling_mode = "mean"

In [ ]:
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=12,
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
target_sampling_rate = feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


In [ ]:
input_column = "file_path"
output_column = "encoded_labels"

In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label for label in examples[output_column]]

    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df).map(
    preprocess_function,
    batch_size=2,
    batched=True,
    # num_proc=4
)
eval_dataset = Dataset.from_pandas(val_df).map(
    preprocess_function,
    batch_size=2,
    batched=True,
    # num_proc=4
)

Map:   0%|          | 0/1411 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

In [ ]:
import pickle as pkl
import os
directory ="/content/drive/MyDrive/MLProject/pickle/"
if not os.path.exists(directory):
    os.makedirs(directory)
# fileObject = open(directory+"train1_dataset", 'wb')
# pkl.dump(train_dataset,fileObject)
#fileObject.close()
fileObject = open(directory+"val1_dataset", 'wb')
pkl.dump(eval_dataset, fileObject)
fileObject.close()

In [ ]:
import pickle as pkl
import os
directory ="/content/drive/MyDrive/MLProject/pickle/"
if not os.path.exists(directory):
    os.makedirs(directory)
fileObject = open(directory+"train1_dataset", 'rb')
train_dataset = pkl.load(fileObject)

fileObject.close()
fileObject = open(directory+"val1_dataset", 'rb')
eval_dataset = pkl.load(fileObject)
fileObject.close()

In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.feature_extractor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(feature_extractor=feature_extractor, padding=True)

In [ ]:
is_regression = False

In [ ]:
import numpy as np
from transformers import EvalPrediction

In [ ]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score
def compute_metrics(p: EvalPrediction):
    print(type(p))
    print(p.predictions[0])
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    #preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
    print(type(preds))
    print(type(p.predictions))
    arr = torch.from_numpy(p.predictions)
    print(type(arr))
    preds = torch.sigmoid(arr).cpu()
    preds = np.round(preds)
    if is_regression:
      return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
      acc =  accuracy_score(p.label_ids,preds)
      return {"accuracy": acc}


In [ ]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-100k-voxpopuli and are newly initialized: ['classifier.out_proj.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

In [ ]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
from transformers import TrainingArguments
model_save_path = '/content/drive/MyDrive/MLProject/weights'
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/MLProject/weights/wav2vec2-base-100k-eating-sound-collection",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-base-100k-eating-sound-collection"
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=True,
    save_steps=20,
    eval_steps=20,
    logging_steps=20,
    learning_rate=1e-4,
    save_total_limit=2,
)

training_args = TrainingArguments(
    # _n_gpu=1,
    adafactor=False,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    bf16=False,
    bf16_full_eval=False,
    dataloader_drop_last=False,
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
    ddp_bucket_cap_mb=None,
    ddp_find_unused_parameters=None,
    debug=[],
    deepspeed=None,
    disable_tqdm=False,
    do_eval=True,
    do_predict=True,
    do_train=True,
    eval_accumulation_steps=None,
    eval_steps=300,
    evaluation_strategy="steps",
    fp16=True,
    fp16_full_eval=False,
    gradient_accumulation_steps=2,
    gradient_checkpointing=False,
    greater_is_better=None,
    group_by_length=False,
    ignore_data_skip=False,
    label_names=None,
    label_smoothing_factor=0.0,
    learning_rate=0.0001,
    load_best_model_at_end=False,
    local_rank=-1,
    max_grad_norm=1.0,
    max_steps=-1,
    metric_for_best_model=None,
    no_cuda=False,
    num_train_epochs=10,
    optim="adamw_hf",
    output_dir=f"{model_save_path}/wav2vec2-base-100k-voxpopuli-gtzan-music",
    overwrite_output_dir=True,
    past_index=-1,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=1,
    prediction_loss_only=False,
    remove_unused_columns=True,
    report_to=['tensorboard'],
    resume_from_checkpoint=None,
    save_on_each_node=False,
    save_steps=100,
    save_total_limit=2,
    seed=42,
    skip_memory_metrics=True,
    tf32=None,
    tpu_metrics_debug=False,
    tpu_num_cores=None,
    use_legacy_prediction_loop=False,
    warmup_ratio=0.0,
    warmup_steps=2000,
    weight_decay=0.0,
)

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=feature_extractor,
)


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:423: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
300,No log,0.477975,0.000000
600,0.481100,0.458548,0.000000
900,0.481100,0.449214,0.000000
1200,0.463700,0.447524,0.019802
1500,0.451100,0.457199,0.000000
1800,0.451100,0.464723,0.000000
2100,0.475000,0.489867,0.000000
2400,0.475000,0.490051,0.000000
2700,0.487400,0.489189,0.000000


<class 'transformers.trainer_utils.EvalPrediction'>
[-1.8964844  -0.6665039  -1.4072266  -1.1650391  -0.85253906 -1.3876953
 -1.9443359  -2.046875   -1.1923828  -0.34936523 -1.2353516  -1.3535156 ]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
<class 'transformers.trainer_utils.EvalPrediction'>
[-2.1152344  -0.2890625  -1.4814453  -1.3984375  -0.69873047 -2.1347656
 -2.0234375  -2.0078125  -1.390625   -0.20715332 -0.8935547  -1.6835938 ]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
<class 'transformers.trainer_utils.EvalPrediction'>
[-2.453125   -0.5527344  -1.4892578  -1.7910156  -0.8276367  -2.7578125
 -2.1425781  -2.2070312  -1.6689453  -0.17993164 -1.1533203  -1.78125   ]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
<class 'transformers.trainer_utils.EvalPrediction'>
[-2.671875   -0.7290039  -1.6435547  -1.4013672  -1.2324219  -2.2539062
 -2.2128906  -2.640625   -1.3212891  -0.39038086 -1.4941406  -1.74902

In [ ]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/kaggle/working/cache
%env HF_DATASETS_CACHE=/kaggle/working/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/kaggle/working/cache
env: HF_DATASETS_CACHE=/kaggle/working/cache
env: CUDA_LAUNCH_BLOCKING=1
